In [2]:
!pip install psycopg2-binary 
!pip install scikit-learn

In [ ]:
from query_API import *

import pandas as pd
import dask.dataframe as dd

In [48]:
history_df = query_from_db("Select roundid, price, updatedat, pair, decimals From chainlink_history")
real_time_df = query_from_db("Select roundid, price, updatedat, pair, decimals From chainlink_real_time")

In [49]:
df = pd.concat([history_df, real_time_df])
df

,roundid,price,updatedat,pair,decimals
0,92233720368547771281,4652109401959,2021-12-31 22:13:16,BTC / USD,8
1,92233720368547771282,4639357085285,2021-12-31 22:55:11,BTC / USD,8
2,92233720368547771283,4635816164458,2021-12-31 23:56:36,BTC / USD,8
3,92233720368547771284,4661544706500,2022-01-01 00:42:51,BTC / USD,8
4,92233720368547771285,4670395911240,2022-01-01 00:56:01,BTC / USD,8
...,...,...,...,...,...
1000,92233720368547792947,2969124500000,2023-04-27 19:57:47,BTC / USD,8
1001,92233720368547792948,2952345020382,2023-04-27 20:12:23,BTC / USD,8
1002,92233720368547792949,2967974870022,2023-04-27 20:39:23,BTC / USD,8
1003,92233720368547792950,2963412366000,2023-04-27 20:56:23,BTC / USD,8


In [50]:
df['price_shift'] = df['price'].shift(-1)

In [51]:
df

,roundid,price,updatedat,pair,decimals,price_shift
0,92233720368547771281,4652109401959,2021-12-31 22:13:16,BTC / USD,8,4639357085285
1,92233720368547771282,4639357085285,2021-12-31 22:55:11,BTC / USD,8,4635816164458
2,92233720368547771283,4635816164458,2021-12-31 23:56:36,BTC / USD,8,4661544706500
3,92233720368547771284,4661544706500,2022-01-01 00:42:51,BTC / USD,8,4670395911240
4,92233720368547771285,4670395911240,2022-01-01 00:56:01,BTC / USD,8,4669785530693
...,...,...,...,...,...,...
1000,92233720368547792947,2969124500000,2023-04-27 19:57:47,BTC / USD,8,2952345020382
1001,92233720368547792948,2952345020382,2023-04-27 20:12:23,BTC / USD,8,2967974870022
1002,92233720368547792949,2967974870022,2023-04-27 20:39:23,BTC / USD,8,2963412366000
1003,92233720368547792950,2963412366000,2023-04-27 20:56:23,BTC / USD,8,2963476675728


In [52]:
ddf = dd.from_pandas(df, npartitions=10)

In [60]:
def percentage_diff(df):
    return (df['price_shift']-df['price']) / df['price'] * 100

In [61]:
# calculate the percentage difference using the map_partitions() method
ddf['percentage_diff'] = ddf.map_partitions(percentage_diff, meta=('x', 'f8'))

# drop the first row, which will have NaN values
ddf = ddf.dropna()

# convert object to float and round to the second decimal points
ddf['percentage_diff'] = ddf['percentage_diff'].astype(float).round(2)

# save dask dataframe to pandas dataframe
df_compute = ddf.compute()

In [62]:
df_compute.sort_values(by=['roundid'])

,roundid,price,updatedat,pair,decimals,price_shift,percentage_diff
0,92233720368547771281,4652109401959,2021-12-31 22:13:16,BTC / USD,8,4639357085285,-0.27
1,92233720368547771282,4639357085285,2021-12-31 22:55:11,BTC / USD,8,4635816164458,-0.08
2,92233720368547771283,4635816164458,2021-12-31 23:56:36,BTC / USD,8,4661544706500,0.55
3,92233720368547771284,4661544706500,2022-01-01 00:42:51,BTC / USD,8,4670395911240,0.19
4,92233720368547771285,4670395911240,2022-01-01 00:56:01,BTC / USD,8,4669785530693,-0.01
...,...,...,...,...,...,...,...
999,92233720368547792946,2970335578388,2023-04-27 19:56:23,BTC / USD,8,2969124500000,-0.04
1000,92233720368547792947,2969124500000,2023-04-27 19:57:47,BTC / USD,8,2952345020382,-0.57
1001,92233720368547792948,2952345020382,2023-04-27 20:12:23,BTC / USD,8,2967974870022,0.53
1002,92233720368547792949,2967974870022,2023-04-27 20:39:23,BTC / USD,8,2963412366000,-0.15


In [59]:
df.price_shift

0       4639357085285
1       4635816164458
2       4661544706500
3       4670395911240
4       4669785530693
            ...      
1000    2952345020382
1001    2967974870022
1002    2963412366000
1003    2963476675728
1004             None
Name: price_shift, Length: 21671, dtype: object

In [358]:
df.roundid.max()

Decimal('92233720368547792946')

In [345]:
roundid = str(df.roundid.max()-1)
query = "SELECT roundid, price, updatedat, pair, decimals FROM chainlink_real_time WHERE roundid > {}"
formatted_query = query.format(roundid)
formatted_query

'SELECT roundid, price, updatedat, pair, decimals FROM chainlink_real_time WHERE roundid > 92233720368547792845'

In [346]:
real_time_compute = query_from_db(formatted_query)
real_time_compute

,roundid,price,updatedat,pair,decimals
0,92233720368547792846,2978420238823,2023-04-26 16:55:23,BTC / USD,8
1,92233720368547792847,2972838000000,2023-04-26 17:55:23,BTC / USD,8
2,92233720368547792848,2971691104355,2023-04-26 18:55:23,BTC / USD,8
3,92233720368547792849,2954604288825,2023-04-26 19:17:59,BTC / USD,8
4,92233720368547792850,2938606090000,2023-04-26 19:18:59,BTC / USD,8
5,92233720368547792851,2922931413440,2023-04-26 19:23:59,BTC / USD,8
6,92233720368547792852,2905813425000,2023-04-26 19:27:59,BTC / USD,8
7,92233720368547792853,2889630359000,2023-04-26 19:35:35,BTC / USD,8
8,92233720368547792854,2874700000000,2023-04-26 19:38:23,BTC / USD,8
9,92233720368547792855,2854452000000,2023-04-26 19:39:23,BTC / USD,8


In [347]:
real_time_ddf = dd.from_pandas(real_time_compute, npartitions=10)

In [348]:
# calculate the percentage difference using the map_partitions() method
real_time_ddf['percentage_diff'] = real_time_ddf['price'].map_partitions(percentage_diff, meta=('x', 'f8'))

# drop the first row, which will have NaN values
# real_time_ddf = real_time_ddf.dropna()

# convert object to float and round to the second decimal points
real_time_ddf['percentage_diff'] = real_time_ddf['percentage_diff'].astype(float).round(2)

# save dask dataframe to pandas dataframe
df = real_time_ddf.compute()
df

,roundid,price,updatedat,pair,decimals,percentage_diff
1,92233720368547792847,2972838000000,2023-04-26 17:55:23,BTC / USD,8,0.19
2,92233720368547792848,2971691104355,2023-04-26 18:55:23,BTC / USD,8,0.04
3,92233720368547792849,2954604288825,2023-04-26 19:17:59,BTC / USD,8,0.58
4,92233720368547792850,2938606090000,2023-04-26 19:18:59,BTC / USD,8,0.54
5,92233720368547792851,2922931413440,2023-04-26 19:23:59,BTC / USD,8,0.54
7,92233720368547792853,2889630359000,2023-04-26 19:35:35,BTC / USD,8,0.56
8,92233720368547792854,2874700000000,2023-04-26 19:38:23,BTC / USD,8,0.52
9,92233720368547792855,2854452000000,2023-04-26 19:39:23,BTC / USD,8,0.71
10,92233720368547792856,2833985935258,2023-04-26 19:40:23,BTC / USD,8,0.72
11,92233720368547792857,2849969441433,2023-04-26 19:41:23,BTC / USD,8,-0.56


In [63]:
test = query_from_db("Select count(roundid) From chainlink_compute")

In [64]:
test

,count
0,21670
